In [352]:
def read_file(file_name):
	with open(file_name, "r") as f:
		data = f.read().splitlines()
	return data

In [353]:
EXAMPLE_FILE = "example"
INPUT_FILE = "input"
left = (-1, 0)
right = (1, 0)
down = (0, -1)
up = (0, 1)
horizontal_offset = 2
cave_width = 7

In [354]:
from itertools import cycle

rock_minus = set([(x, 0) for x in range(4)])
rock_plus = set([(x, 1) for x in range(3)] + [(1, 0), (1 , 2)])
rock_corner = set([(x, 0) for x in range(3)] + [(2, y) for y in range(1, 3)])
rock_column = set([(0, y) for y in range(4)])
rock_sqaure = set([(x, y) for x in range(2) for y in range(2)])

def init_cave(file_name):
	cave = set()
	jets = cycle(list(read_file(file_name)[0]))
	rocks = cycle([rock_minus, rock_plus, rock_corner, rock_column, rock_sqaure])
	return cave, jets, rocks


In [355]:
def move_rock(rock, direction, steps):
	new_rock = []
	for x, y in rock:
		x += direction[0] * steps
		y += direction[1] * steps
		if x not in range(cave_width) or y < 0 or (x, y) in cave:
			return False
		new_rock.append((x, y))
	new_rock = set(new_rock)
	rock.clear()
	rock.update(new_rock)
	return True

In [356]:
def spawn_rock(x, y):
	spawn = next(rocks).copy()
	move_rock(spawn, up, y)
	move_rock(spawn, right, x)
	return spawn

In [357]:
def move_by_jet(rock, jet):
	direction = left if jet == '<' else right
	move_rock(rock, direction, 1)

In [358]:
from PIL import Image, ImageOps
import numpy as np

IMG_SCALE_FACTOR = 80

def render_cave(y):
	data = np.zeros((y+1,cave_width,3), dtype=np.uint8 )
	for coords in cave:
		color = [255,255,255]
		data[coords[1], coords[0]] = color
	img = Image.fromarray(data)
	img = img.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)
	img = ImageOps.scale(img, IMG_SCALE_FACTOR, Image.Resampling.BOX)
	display(img) 

In [ ]:
def solve_part1(jets):
	count = 0
	y = 0
	while count < 2022:
		if count % 1_000 == 0:
			print(count)
		rock = spawn_rock(horizontal_offset, y + 3)
		moving = True
		while moving:
			move_by_jet(rock, next(jets))
			moving = move_rock(rock, down, 1)
		cave.update(rock)
		y = max([y for x, y in cave]) + 1
		count += 1
	return y

# cave, jets, rocks = init_cave(EXAMPLE_FILE)
# print(solve_part1(jets))
cave, jets, rocks = init_cave(INPUT_FILE)
solve_part1(jets)